# Remapping PBRT roughness to LiAR roughness

PBRT remaps roughness to alpha as following [1]:

[1] https://github.com/mmp/pbrt-v3/blob/13d871faae88233b327d04cda24022b8bb0093ee/src/core/microfacet.h#L83-L87

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

pbrt_roughness = np.arange(
    0.001, 1, 0.001, dtype=np.float32
)  # we start from 0.001 to avoid log(0)

x = np.log(pbrt_roughness)
pbrt_alpha = (
    1.62142 + 0.819955 * x + 0.1734 * x**2 + 0.0171201 * x**3 + 0.000640711 * x**4
)

fig, ax = plt.subplots()
ax.plot(pbrt_roughness, pbrt_alpha, label="pbrt_alpha")
ax.set_xlabel("PBRT roughness")
ax.set_ylabel("alpha")
ax.legend()
plt.show()

print(f"min: {pbrt_alpha[0]}")

According to [2], this seems to be a fit of `1.62142 * sqrt(pbrt_roughness)`

[2] https://computergraphics.stackexchange.com/q/12168

In [ ]:
fig, ax = plt.subplots()
ax.plot(pbrt_roughness, pbrt_alpha, label="pbrt_alpha")
ax.plot(
    pbrt_roughness,
    1.62142 * np.sqrt(pbrt_roughness),
    label="1.62142 * sqrt(pbrt_roughness)",
)
ax.set_xlabel("PBRT roughness")
ax.set_ylabel("alpha")
ax.legend()
plt.show()

print(f"min: {1.62142 * np.sqrt(np.min(pbrt_roughness))}")

It's not even a good fit, as we can do much better:

In [ ]:
from numpy.polynomial import Polynomial

domain = window = [min(x), max(x)]

p = Polynomial.fit(
    x, 1.62142 * np.sqrt(pbrt_roughness), 4, domain=domain, window=window
)
print(p)

fig, ax = plt.subplots()
# ax.plot(pbrt_roughness, pbrt_alpha, label=f"pbrt_alpha")
ax.plot(
    pbrt_roughness,
    1.62142 * np.sqrt(pbrt_roughness),
    label="1.62142 * sqrt(pbrt_roughness)",
)
ax.plot(pbrt_roughness, p(x), label="4th degree polyomial fit")
ax.set_xlabel("PBRT roughness")
ax.set_ylabel("alpha")
ax.legend()
plt.show()

print(f"min: {p(x[0])}")

But LiAR actually uses the common mapping `alpha = liar_roughness ** 2` [3,4,5]

So we take PBRT's RoughnessToAlpha, take the square root of that, and we fit a new polynomial:

- [3] B. Burley 2012, Physically Based Shading at Disney
- [4] http://graphicrants.blogspot.com/2013/08/specular-brdf-reference.html
- [5] B. Karis 2013, Real Shading in Unreal Engine 4

In [ ]:
from functools import reduce

pbrt_to_liar_roughness = np.sqrt(pbrt_alpha)

p = Polynomial.fit(x, pbrt_to_liar_roughness, 4, domain=domain, window=window)
print(p)

fig, ax = plt.subplots()
ax.plot(pbrt_roughness, pbrt_to_liar_roughness, label=f"pbrt_to_liar_roughness")
ax.plot(pbrt_roughness, p(x), label="4th degree polyomial fit")
ax.set_xlabel("PBRT roughness")
ax.set_ylabel("LiAR roughness")
ax.legend()
plt.show()

print(f"min pbrt_to_liar_roughness: {pbrt_to_liar_roughness[:5]}")
print(f"min fit: {p(x[0:5])}")

factors = [f"{x!r}f" for x in p.coef]
print(reduce(lambda acc, f: f"{f} + ({acc}) * x", reversed(factors)))